In [2]:
import pandas as pd


학습 및 테스트 데이터 읽어오기

In [5]:
dftrain = pd.read_csv('./data/train_job/train.csv')
dftest = pd.read_csv('./data/test_job.csv')

test = dftest.copy()

학습 데이터 확인해보기

In [8]:
dftrain.head()

,userID,jobID,applied
0,fe292163d06253b716e9a0099b42031d,15de21c670ae7c3f6f3f1f37029303c9,0
1,6377fa90618fae77571e8dc90d98d409,55b37c5c270e5d84c793e486d798c01d,0
2,8ec0888a5b04139be0dfe942c7eb4199,0fcbc61acd0479dc77e3cccc0f5ffca7,1
3,f862b39f767d3a1991bdeb2ea1401c9c,3b5dca501ee1e6d8cd7b905f4e1bf723,0
4,cac14930c65d72c16efac2c51a6b7f71,287e03db1d99e0ec2edb90d079e142f3,0


데이터가 암호화 되어 있어서, 이 암호화된 string을 정수로 바꿔줘야 할 것 같다.

학습 및 테스트 데이터를 변환하기 전에, `userID`와 `jobID`에 관련된 추가 정보들이 담긴 `csv`파일들을 더 불러온다.

In [14]:
df_job_companies = pd.read_csv('./data/train_job/job_companies.csv')
df_job_tags = pd.read_csv('./data/train_job/job_tags.csv')
df_tags = pd.read_csv('./data/train_job/tags.csv')
df_user_tags = pd.read_csv('./data/train_job/user_tags.csv')

In [17]:
print('job_companies.csv : ', df_job_companies.columns)
print('job_tags.csv : ', df_job_tags.columns)
print('tags.csv : ', df_tags.columns)
print('user_tags.csv : ', df_user_tags.columns)


job_companies.csv :  Index(['companyID', 'jobID', 'companySize'], dtype='object')
job_tags.csv :  Index(['jobID', 'tagID'], dtype='object')
tags.csv :  Index(['tagID', 'keyword'], dtype='object')
user_tags.csv :  Index(['userID', 'tagID'], dtype='object')


In [25]:
print(df_job_companies.isna().sum())

companyID       0
jobID           0
companySize    90
dtype: int64


* 90개의 회사가 `companySize` 데이터가 존재하지 않음(`NaN`)
* `jobID`는 `tagID`, `companyID`, `companySize`, `tagID`와 관련이 있으며, `userID`는 `tagID`와 관련이 있음

In [26]:
dftrain.userID = pd.Categorical(dftrain.userID)
dftrain['code'] = dftrain.userID.cat.codes
# dftrain
